In [1]:
import numpy as np
import yroot
import yleaf
from ytool import *

In [4]:
modenames = [
            "y01605",
            "y04466",
            "y05420",
            "y05427",
            "y06098",
            "y07206",
            "y10002",
            "y17891",
            "y24954",
            "y29172",
            "y29176",
            "y35663",
            "y36413",
            "y36415",
            "y39990",
            "y49096"
            ]

In [8]:
a = "sdfsadsasadatempasd"
"temb" in a

False

In [16]:
for mode in modenames:
    repo = f"/storage3/Clusters/YoungTree/{mode[1:]}/"
    files = os.listdir(repo)
    files = [file for file in files if "temp" in file]
    if len(files)>0:
        print(mode, files)
        print(files[0][6:11])

y01605 ['ytree_00107_temp.pickle', 'ytree_00108_temp.pickle', 'ytree_00103_temp.pickle', 'ytree_00101_temp.pickle', 'ytree_00106_temp.pickle', 'ytree_00105_temp.pickle', 'ytree_00104_temp.pickle', 'ytree_00110_temp.pickle', 'ytree_00109_temp.pickle', 'ytree_00111_temp.pickle', 'ytree_00102_temp.pickle']
00107
y04466 ['ytree_00025_temp.pickle', 'ytree_00024_temp.pickle', 'ytree_00023_temp.pickle', 'ytree_00022_temp.pickle', 'ytree_00020_temp.pickle', 'ytree_00021_temp.pickle']
00025
y05427 ['ytree_00008_temp.pickle', 'ytree_00014_temp.pickle', 'ytree_00013_temp.pickle', 'ytree_00012_temp.pickle', 'ytree_00010_temp.pickle', 'ytree_00011_temp.pickle']
00008
y06098 ['ytree_00015_temp.pickle', 'ytree_00014_temp.pickle', 'ytree_00013_temp.pickle', 'ytree_00012_temp.pickle', 'ytree_00011_temp.pickle', 'ytree_00010_temp.pickle']
00015
y07206 ['ytree_00015_temp.pickle', 'ytree_00014_temp.pickle', 'ytree_00013_temp.pickle', 'ytree_00012_temp.pickle', 'ytree_00011_temp.pickle', 'ytree_00010_temp.

In [19]:
modes = ["y04466",
"y05427",
"y06098",
"y07206"]

In [14]:
files

[]

In [20]:
modes

['y04466', 'y05427', 'y06098', 'y07206']

In [33]:
for mode in modes:
    resultdir = f"/storage3/Clusters/YoungTree/{mode[1:]}/"
    files = os.listdir(resultdir)
    files = [file for file in files if file.endswith("pickle")]
    fnames = [file for file in files if "temp" in file]
    # print(fnames)
    # raise ValueError("stop")
    for fname in fnames:
        iout = int(fname[6:11])
        file, _ = pklload(f"{resultdir}{fname}")
        if isinstance(file, dict):
            keys = list(file.keys())
            count = 0
            for key in keys:
                gals = file[key]['gal'] if count==0 else np.hstack((gals, file[key]['gal']))
                count += 1
            pklsave(gals, f"{resultdir}ytree_{iout:05d}.pickle", overwrite=True)
            print(f"Save `{resultdir}ytree_{iout:05d}.pickle`")
            # os.remove(fname)
            # print(f"Remove `{resultdir}ytree_{iout:05d}_temp.pickle`")
        del gals

Save `/storage3/Clusters/YoungTree/04466/ytree_00025.pickle`
Save `/storage3/Clusters/YoungTree/04466/ytree_00024.pickle`
Save `/storage3/Clusters/YoungTree/04466/ytree_00023.pickle`
Save `/storage3/Clusters/YoungTree/04466/ytree_00022.pickle`
Save `/storage3/Clusters/YoungTree/04466/ytree_00020.pickle`
Save `/storage3/Clusters/YoungTree/04466/ytree_00021.pickle`
Save `/storage3/Clusters/YoungTree/05427/ytree_00008.pickle`
Save `/storage3/Clusters/YoungTree/05427/ytree_00014.pickle`
Save `/storage3/Clusters/YoungTree/05427/ytree_00013.pickle`
Save `/storage3/Clusters/YoungTree/05427/ytree_00012.pickle`
Save `/storage3/Clusters/YoungTree/05427/ytree_00010.pickle`
Save `/storage3/Clusters/YoungTree/05427/ytree_00011.pickle`
Save `/storage3/Clusters/YoungTree/06098/ytree_00015.pickle`
Save `/storage3/Clusters/YoungTree/06098/ytree_00014.pickle`
Save `/storage3/Clusters/YoungTree/06098/ytree_00013.pickle`
Save `/storage3/Clusters/YoungTree/06098/ytree_00012.pickle`
Save `/storage3/Clusters

(153, 0, 35)

In [18]:
a = 3
b = a
c = a

In [20]:
gc.get_count(a)

TypeError: gc.get_count() takes no arguments (1 given)

In [30]:
import weakref

In [35]:
class test:
    def __init__(self, val):
        self.val = np.arange(1000000)
        self.name = f"{val}"

In [59]:
insts = [test(0), test(1), test(2), test(3)]

In [68]:
dict_test = {}

In [69]:
for i in range(4):
    dict_test[i] = weakref.ref(test(i))

In [70]:
dict_test

{0: <weakref at 0x7fecd875e8e0; dead>,
 1: <weakref at 0x7fecd8ada250; dead>,
 2: <weakref at 0x7fecd8760450; dead>,
 3: <weakref at 0x7fecd87604a0; dead>}

In [61]:
for inst in insts:
    dict_test[inst.name] = weakref.ref(inst)
    # dict_test[inst.name] = inst

In [62]:
insts[0].val = 3

In [63]:
dict_test

{'0': <__main__.test at 0x7fecd8a77520>,
 '1': <__main__.test at 0x7fecd8a5a140>,
 '2': <__main__.test at 0x7fecd8a59d80>,
 '3': <__main__.test at 0x7fecd8a58550>}

In [64]:
del insts

In [65]:
dict_test

{'0': <__main__.test at 0x7fecd8a77520>,
 '1': <__main__.test at 0x7fecd8a5a140>,
 '2': <__main__.test at 0x7fecd8a59d80>,
 '3': <__main__.test at 0x7fecd8a58550>}

In [44]:
del insts

In [45]:
dict_test

{'0': <weakref at 0x7fecd8712160; to 'test' at 0x7fecd8a59ea0>,
 '1': <weakref at 0x7fecd889dc60; dead>,
 '2': <weakref at 0x7fecd889df30; dead>,
 '3': <weakref at 0x7fecd889d990; to 'test' at 0x7fed4b613700>}

In [51]:
dict_test['1']

In [22]:
a = test(None)
b = test(None)

In [29]:
test.__init__.__defaults__

In [23]:
id(a), id(b)

(140655235092576, 140655235092768)

In [24]:
a == b

False

In [25]:
a is b

False

In [76]:
del a

In [11]:
print(MB())
time.sleep(1)
a = test(3)
time.sleep(1)
print(MB())

243.35546875
259.15234375


In [12]:
print(MB())
d = {}
print(MB())
d[1] = test(4)
print(MB())
d[2] = a
print(MB())

259.15234375
259.15234375
279.69140625
279.69140625


In [14]:
print(MB())
d[1].val = d[1].val/2
print(MB())

279.69140625
279.91796875


In [15]:
print(MB())
del d[1]
print(MB())

279.91796875
264.65625


In [56]:
d = {}
d[1] = a
d

{1: <__main__.test at 0x7fa1b1482230>}

In [57]:
del d[1]

In [58]:
d

{}

In [59]:
a

In [60]:
gc.collect()

2291

In [61]:
a

In [50]:
locals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', 'sys', 'os', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', '_i2', 'np', 'a', '_2', '_i3', 'yroot', 'yleaf', 'time', 'psutil', 'logging', 'pickle', 'nb', 'Iterable', 'inspect', 'traceback', 'importlib', 'params', 'ncpu', 'DebugDecorator', 'threading', 'DisplayCPU', 'DotDict', 'memory_tracker', 'timer', 'plot', 'make_logname', 'RotatingFileHandler', 'custom_debugger', 'dprint_', 'mode2repo', 'load_nout', 'load_nstep', 'out2step', 'step2out', 'ioutistep', 'printgal', 'MB', 'GB', 'pklsave', 'pklload', 'distance3d', 'cut_sphere', 'howmany', 'printtime', 'rms', 'nbnorm', 'nbsum', 'set_num_threads', 'large_isin', 'large_isind', 'atleast_numba', 'atleast_numba_para', 'atleast_isin', 'FortranFile', 'read_galaxymaker', 'skipread_i', 'skipread_f', '_i4', '_4', '_i5', '_5', '_i6', 'data', '_i7', '_i8', '_8',

In [48]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', 'sys', 'os', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', '_i2', 'np', 'a', '_2', '_i3', 'yroot', 'yleaf', 'time', 'psutil', 'logging', 'pickle', 'nb', 'Iterable', 'inspect', 'traceback', 'importlib', 'params', 'ncpu', 'DebugDecorator', 'threading', 'DisplayCPU', 'DotDict', 'memory_tracker', 'timer', 'plot', 'make_logname', 'RotatingFileHandler', 'custom_debugger', 'dprint_', 'mode2repo', 'load_nout', 'load_nstep', 'out2step', 'step2out', 'ioutistep', 'printgal', 'MB', 'GB', 'pklsave', 'pklload', 'distance3d', 'cut_sphere', 'howmany', 'printtime', 'rms', 'nbnorm', 'nbsum', 'set_num_threads', 'large_isin', 'large_isind', 'atleast_numba', 'atleast_numba_para', 'atleast_isin', 'FortranFile', 'read_galaxymaker', 'skipread_i', 'skipread_f', '_i4', '_4', '_i5', '_5', '_i6', 'data', '_i7', '_i8', '_8',

In [46]:
gc.garbage

[]

In [45]:
gc.get_referents(e)[0]

__main__.test

In [30]:
gc.get_debug()

0

In [11]:
data = pklload("/storage3/Clusters/24954/YoungTree/ytree_00157_temp.pickle")

In [3]:
isind = large_isind

In [4]:
isind

CPUDispatcher(<function large_isind at 0x7fd436cb4ca0>)

In [7]:
from numba.core.config import NUMBA_NUM_THREADS
NUMBA_NUM_THREADS

80

In [2]:
import numpy as np
a = np.array([1])
np.percentile(a, q=[16,50,84])

array([1., 1., 1.])

In [73]:
def a1(a,b):
    return np.sum(a*b)

def a2(a,b):
    return np.convolve( a,b[::-1], mode='valid' )[0]

import numba as nb
@nb.njit(fastmath=True)
def a3(a:np.ndarray,b:np.ndarray)->float:
    n:int = len(a)
    s:float = 0.
    for i in nb.prange(n):
        s += a[i]*b[i]
    return s

@nb.jit(parallel=True)
def a4(a,b):
    n = len(a)
    result = np.full(n, 0., dtype=np.float64)
    for i in nb.prange(n):
        result[i] = a[i]*b[i]
    return np.sum(result)

In [35]:
a = np.random.rand(100000)
b = np.random.rand(100000)

In [72]:
a1(a,b), a2(a,b), a3(a,b)

(24919.238693341762, 24919.23869334176, 24919.238693341747)

In [49]:
%timeit -n 10000 -r 3 a1(a,b)

119 µs ± 435 ns per loop (mean ± std. dev. of 3 runs, 10,000 loops each)


In [47]:
%timeit -n 10000 -r 3 a2(a,b)

17.9 µs ± 196 ns per loop (mean ± std. dev. of 3 runs, 10,000 loops each)


In [76]:
%timeit -n 10000 -r 3 a3(a,b)

53.9 µs ± 130 ns per loop (mean ± std. dev. of 3 runs, 10,000 loops each)


In [10]:
np.convolve( a,b[::-1], mode='valid' )[0]

18

In [9]:
import psutil
import random
import threading
import time

class DisplayCPU(threading.Thread):
    def __enter__(self):
        print("enter")
        self.start()
        self.ref = time.time()
        print("enter end")

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("exit")
        self.stop()
        print("exit end")

    def run(self):
        print("run")
        self.go = False
        self.running = True

        currentProcess = psutil.Process()

        while self.running and (time.time()-self.ref < 10):
            print(currentProcess.cpu_percent(interval=1))
        print("run end")

    def stop(self):
        print("stop")
        self.ref = -1
        self.running = False
        print("stop end")

# ----

def i_hate_this():
    print("hate")
    tab = []
    for i in range(1000000):
        tab.append(random.randint(1, 10000))
    tab.sort()
    print("hate end")
    return tab

In [11]:
i_hate_this.__name__

'i_hate_this'

In [16]:
import numpy as np
isind = lambda a,b: np.where(np.isin(a,b))[0]

In [18]:
isin = np.isin
isin(np.arange(10), np.array([1,2,3]))

array([False,  True,  True,  True, False, False, False, False, False,
       False])

In [17]:
isind(np.arange(10), np.array([1,2,3]))

array([1, 2, 3])

In [26]:
display_cpu = DisplayCPU()

display_cpu.start()
try:
    result = i_hate_this()
finally: # stop thread even when I press Ctrl+C
    display_cpu.stop()

97.9
84.0
78.5
84.0
112.2
115.3
86.9
98.0
98.2
83.5
98.2
118.9


In [10]:
display_cpu = DisplayCPU()
display_cpu.ref = time.time()
with display_cpu:
    i_hate_this()

enter
run
enter end
hate
hate end
exit
stop
stop end
exit end


70.1
run end


In [4]:
from rur import uri
from rur.sci import photometry

In [5]:
backups = pklload("/storage3/Clusters/07206/YoungTree/ytree_00117.pickle")

In [7]:
backups.shape

(392,)

In [9]:
backups[0], backups[0].dtype

((135956, 1, 117, 0.65000004, 6.341491e+10, 0.22753587, 0.38074753, 0.5141741, 197.13655, 215.45401, 54.09123, 0.000851, 0.00040146, 6.051862e+10, array([[116,   1],
        [115,   1],
        [114,   1],
        [113,   3],
        [113, 366],
        [112,   2],
        [112, 360]]), array([[3.77491877, 0.95820116, 0.94262481, 0.8853901 , 0.98870271],
        [3.67953292, 0.94513646, 0.91963217, 0.8340207 , 0.98074359],
        [3.52424307, 0.94355837, 0.89675284, 0.71883164, 0.96510023],
        [3.18660495, 0.8226217 , 0.87902973, 0.49747592, 0.9874776 ],
        [1.73318215, 0.10988932, 0.96580602, 0.23057275, 0.42691407],
        [2.83970921, 0.73514534, 0.87760714, 0.28467841, 0.94227833],
        [1.89516662, 0.18489239, 0.93958878, 0.22794081, 0.54274464]]), array([[118,   1],
        [119,   1],
        [120,   1],
        [120, 323],
        [121,   1],
        [121, 320],
        [122,   1],
        [122, 318]]), array([[3.80362497, 0.94360824, 0.95787307, 0.91105617, 0.99

In [10]:
backup = pklload("/storage3/Clusters/07206/YoungTree/ytree_00010_temp.pickle")

In [27]:
import sys
sys.getsizeof(backup[0][1]['saved']['matchrate'][15][1][1])

112

In [29]:
backup[0][1]['saved']['matchrate'][15]

{1: (0.8966794973880926,
  array([ True,  True,  True,  True, False,  True,  True,  True, False,
          True,  True,  True,  True,  True,  True,  True, False,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
         False, False,  True,  True,  True,  True, False,  True, False,
          True,  True,  True,  True,  True,  True, False,  True,  True,
          True,  True,  True,  True,  True,  True,  True, False, False,
          True,  True,  True,  True,  True,  True, False, False, False,
          True,  True,  True,  True,  True,  True, False,  True,  True,
         False, False, False,  True,  True, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False,  True,  True, False,  True, False,  True,  True,
          True,  True,  True,  True, False, False,  True,  True, False,
          True,  True,  True, False,  True,  True, False,  True,  True,
          True,  True,  True,  True,  T